In [62]:
import numpy as np
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import urllib
import openpyxl
import xlwings as xw
from xlwings.constants import DeleteShiftDirection
import fnmatch
import re 
import data_processing as dp
from buffet import prepare
from openpyxl.worksheet.datavalidation import DataValidation
from datetime import date
pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Connecting to SQL database

In [155]:
#Connecting to database
sql_conn = pyodbc.connect('DRIVER={SQL Server};\
                           SERVER=aupanalytics02;\
                           DATABASE=RVDataMart;\
                           Trusted_Connection=yes')
sql_conn2 = pyodbc.connect('DRIVER={SQL Server};\
                           SERVER=aupanalytics02;\
                           DATABASE=RVMonthlyReporting;\
                           Trusted_Connection=yes')

In [156]:
vehicles_base_query = "select vehicles.cVehicleCode as nvic\
    ,vehicles.vcMake \
    ,vehicles.vcModel \
    ,vehicles.vcSeries \
    ,vehicles.vcVariant\
    ,vehicles.vcBodyStyle\
    ,vehicles.vcTransmissionType\
    ,vehicles.vcDriveType\
    ,vehicles.vcFuelType\
    ,vehicles.vcEngineSize\
    ,vehicles.cCylinders\
    ,vehicles.dtReleaseDate\
    ,vehicles.mRetailPrice\
    from RVDataMart.dbo.tblVehicleData as vehicles \
where \
vehicles.vcMake in ('TOYOTA','LEXUS','MAZDA') \
and vehicles.dtReleaseDate >= '15 Feb 2014' \
and vehicles.cVehicleSource = 'G'"

disposals_base_query = "select vehicles.cVehicleCode as nvic\
    ,vehicles.vcMake \
    ,vehicles.vcModel \
    ,vehicles.vcSeries \
    ,vehicles.vcVariant\
    ,vehicles.vcBodyStyle\
    ,vehicles.vcTransmissionType\
    ,vehicles.vcDriveType\
    ,vehicles.vcFuelType\
    ,vehicles.vcEngineSize\
    ,vehicles.cCylinders\
    ,vehicles.dtReleaseDate\
    ,vehicles.mRetailPrice\
    from RVDataMart.dbo.tblVehicleData as vehicles \
where \
vehicles.vcMake in ('TOYOTA','LEXUS','MAZDA') \
and vehicles.dtReleaseDate >= '01 Jan 2013' \
and vehicles.cVehicleSource = 'G'"

vehicles_rv_query = "select link_id.nvic, \
       vehicles.vcMake, \
       vehicles.vcModel, \
       vehicles.vcSeries, \
       vehicles.vcVariant, \
       vehicles.vcBodyStyle, \
       vehicles.vcTransmissionType, \
       vehicles.vcDriveType, \
       vehicles.vcFuelType, \
       vehicles.vcEngineSize, \
       vehicles.cCylinders, \
       link_id.miles_id, \
       vehicles.mRetailPrice as glass_mRetailPrice, \
       miles_vehicles.mRetailPrice as miles_mRetailPrice, \
       rv.Term, \
       rv.Kilometer, \
       rv.RVAmount, \
       rv.rvpercent as rv_system, \
       rv.dtEffective as rv_date \
       from RVDataMart.dbo.tblVehicleData as vehicles \
left join RVMonthlyReporting.dbo.nvic_miles_id as link_id on vehicles.cVehicleCode = link_id.nvic \
left join RVDataMart.dbo.tblVehicleData as miles_vehicles on link_id.miles_id = miles_vehicles.cVehicleCode \
left join RVDataMart.dbo.ResidualValueExtract1 as rv on link_id.miles_id = rv.cVehicleCode \
where link_id.nvic is not NULL \
and vehicles.vcMake in ('TOYOTA','LEXUS','MAZDA') \
and vehicles.dtReleaseDate >= '15 Feb 2014' \
and ((rv.Term = 12 and rv.Kilometer = 10000) \
or (rv.Term = 12 and rv.Kilometer = 20000) \
or (rv.Term = 12 and rv.Kilometer = 30000) \
or (rv.Term = 24 and rv.Kilometer = 20000) \
or (rv.Term = 24 and rv.Kilometer = 40000) \
or (rv.Term = 24 and rv.Kilometer = 60000) \
or (rv.Term = 36 and rv.Kilometer = 30000) \
or (rv.Term = 36 and rv.Kilometer = 60000) \
or (rv.Term = 36 and rv.Kilometer = 90000) \
or (rv.Term = 48 and rv.Kilometer = 40000) \
or (rv.Term = 48 and rv.Kilometer = 80000) \
or (rv.Term = 48 and rv.Kilometer = 120000) \
or (rv.Term = 60 and rv.Kilometer = 50000) \
or (rv.Term = 60 and rv.Kilometer = 100000) \
or (rv.Term = 60 and rv.Kilometer = 150000)) \
and rv.inUse = 'y'"

### Creating vehicles table & RV table

In [157]:
df_vehicles_base = pd.read_sql(vehicles_base_query, sql_conn)
df_vehicles_rvs = pd.read_sql(vehicles_rv_query, sql_conn)
df_disposals_base = pd.read_sql(disposals_base_query, sql_conn)

In [158]:
df_vehicles = df_vehicles_base.copy()
df_disposals= df_disposals_base.copy()

# Data cleaning

In [173]:
df_vehicles['nvic'] = '*' + df_vehicles['nvic'].str.strip()
df_vehicles_rvs['nvic'] = '*' + df_vehicles_rvs['nvic'].str.strip()

#### Cleaning Landcruiser models

In [174]:
for index, row in df_vehicles.iterrows():
    if row['vcModel'] == 'LANDCRUISER':
        if 'PRADO' in row['vcVariant']:
            df_vehicles.loc[index,'vcModel'] = 'LANDCRUISER PRADO'
            next
        elif ('200' in row['vcVariant'] or (row['vcSeries'][3:4]) == '2' and row['vcSeries'][4:5] == '0'):
            df_vehicles.loc[index,'vcModel'] = 'LANDCRUISER 200'
            next
        elif '7' in row['vcSeries'][:4]:
            df_vehicles.loc[index,'vcModel'] = 'LANDCRUISER 70 SERIES'
        else:
            next
    else:
        next

#### make_model

In [175]:
df_vehicles['make_model'] = df_vehicles['vcMake'] + '_' + df_vehicles['vcModel']

#### nvic_family & nvic_year

In [176]:
nvic_vehicles_family = []
nvic_vehicles_year = []
for index, row in df_vehicles.iterrows():
    nvic = str(row['nvic']).strip()
    nvic_end = nvic[-2:]
    'print(nvic[-2:])'
    try:
        nvic_year_val = int(nvic_end)
        nvic_family_val = nvic[:-2]
        
    except:
        try:
            nvic_end = nvic[-3:-1]
            nvic_year_val = int(nvic_end)
            
            nvic_family_val = nvic[:-3] + nvic[-1:] 
        except:
            nvic_year_val = 'NA'
            nvic_family_val = nvic
            
            
    nvic_vehicles_family.append(nvic_family_val)
    nvic_vehicles_year.append(nvic_year_val)
    
df_vehicles['nvic_family'] = nvic_vehicles_family
df_vehicles['nvic_year'] = nvic_vehicles_year

#### make_model_nvic_family

In [177]:
df_vehicles['make_model_nvic_family'] = df_vehicles['vcMake'] + '_' + df_vehicles['vcModel'] + '_' + df_vehicles['nvic_family']

#### Variant cleaning

In [178]:
df_vehicles = dp.vehicle_cleaning_variantkey(df_vehicles)

#### RVs

In [179]:
df_vehicles_uniques = df_vehicles.copy()
df_vehicles_uniques = df_vehicles_uniques[['nvic']]
df_rvs = df_vehicles_rvs[['nvic', 'Term', 'Kilometer', 'RVAmount', 'rv_system', 'rv_date']].copy()

In [180]:
# Creating the RV_xx_xx columns and populating them in the dataframe
term_kkms = [[12, 10000],
             [12, 20000],
             [12, 30000],
             [24, 20000],
             [24, 40000],
             [24, 60000],
             [36, 30000],
             [36, 60000], 
             [36, 90000], 
             [48, 40000],
             [48, 80000],
             [48, 120000],
             [60, 50000],
             [60, 100000],
             [60, 150000]]

term_kkm_count = 0
loop_count = 0
df_vehicles_final = None
for term_kkm in term_kkms:
    term_band = str(term_kkm[0]) + '_' + str(int(term_kkm[1]/1000))
    df_vehicles_temp = df_rvs[(df_rvs['Term'] == term_kkm[0]) & 
                              (df_rvs['Kilometer'] == term_kkm[1])][['nvic', 'RVAmount', 'rv_system']]
    df_vehicles_temp.columns = ['nvic', 'RVAmount_' + term_band, 'RV_system_' + term_band]

    if term_kkm_count == 0:
        df_vehicles_final = pd.merge(df_vehicles_uniques, df_vehicles_temp, how='left', on='nvic')
    else:
        df_vehicles_final = pd.merge(df_vehicles_final, df_vehicles_temp, how='left', on='nvic')
        
    df_vehicles_final = df_vehicles_final.dropna()
    df_vehicles_final = df_vehicles_final.reset_index(drop=True)
    term_kkm_count += 1
    
    #loop_count += 1
    #if loop_count >= 10:
    #    break
df_vehicles_rv_band = df_vehicles_final

In [181]:
df_vehicles = df_vehicles.dropna()

In [182]:
df_vehicles_rv_band.drop_duplicates()

,nvic,RVAmount_12_10,RV_system_12_10,RVAmount_12_20,RV_system_12_20,RVAmount_12_30,RV_system_12_30,RVAmount_24_20,RV_system_24_20,RVAmount_24_40,RV_system_24_40,RVAmount_24_60,RV_system_24_60,RVAmount_36_30,RV_system_36_30,RVAmount_36_60,RV_system_36_60,RVAmount_36_90,RV_system_36_90,RVAmount_48_40,RV_system_48_40,RVAmount_48_80,RV_system_48_80,RVAmount_48_120,RV_system_48_120,RVAmount_60_50,RV_system_60_50,RVAmount_60_100,RV_system_60_100,RVAmount_60_150,RV_system_60_150
0,*000F17,10740.7355,0.72753,10248.0558,0.69550,9801.9830,0.66650,9882.8914,0.67176,9014.5877,0.61531,8383.7792,0.57430,8828.1600,0.60319,7810.0373,0.53700,7062.6347,0.48841,7702.3646,0.53000,6545.6517,0.45480,5724.2625,0.40140,6802.5281,0.47150,5644.7385,0.39623,4838.2697,0.34380
1,*000F18,10740.7355,0.72753,10248.0558,0.69550,9801.9830,0.66650,9882.8914,0.67176,9014.5877,0.61531,8383.7792,0.57430,8828.1600,0.60319,7810.0373,0.53700,7062.6347,0.48841,7702.3646,0.53000,6545.6517,0.45480,5724.2625,0.40140,6802.5281,0.47150,5644.7385,0.39623,4838.2697,0.34380
2,*000G17,9309.0795,0.69753,8860.9506,0.66550,8455.2142,0.63650,8528.8064,0.64176,7739.0195,0.58531,7165.2523,0.54430,7569.4497,0.57319,6643.3914,0.50700,5963.5731,0.45841,6545.4550,0.50000,5493.3386,0.42480,4746.2240,0.37140,5726.9868,0.44150,4673.8910,0.36623,3940.3476,0.31380
3,*000G18,9309.0795,0.69753,8860.9506,0.66550,8455.2142,0.63650,8528.8064,0.64176,7739.0195,0.58531,7165.2523,0.54430,7569.4497,0.57319,6643.3914,0.50700,5963.5731,0.45841,6545.4550,0.50000,5493.3386,0.42480,4746.2240,0.37140,5726.9868,0.44150,4673.8910,0.36623,3940.3476,0.31380
4,*000H17,12341.6010,0.71753,11770.5936,0.68550,11253.6028,0.65650,11347.3742,0.66176,10341.0248,0.60531,9609.9285,0.56430,10124.9583,0.59319,8944.9713,0.52700,8078.7442,0.47841,8820.1804,0.52000,7479.5697,0.44480,6527.5935,0.39140,7777.2851,0.46150,6435.4265,0.38623,5500.7427,0.33380
5,*000I17,10850.4906,0.68753,10324.0340,0.65550,9847.3795,0.62650,9933.8348,0.63176,9006.0023,0.57531,8331.9471,0.53430,8806.7936,0.56319,7718.8709,0.49700,6920.2282,0.44841,7603.8164,0.49000,6367.8021,0.41480,5490.1005,0.36140,6642.2893,0.43150,5405.1245,0.35623,4543.3662,0.30380
6,*000J17,13822.4131,0.69253,13162.3039,0.66050,12564.6403,0.63150,12673.0441,0.63676,11509.6610,0.58031,10664.4822,0.53930,11259.8788,0.56819,9895.7632,0.50200,8894.3675,0.45341,9751.4996,0.49500,8201.6960,0.41980,7101.1706,0.36640,8545.8678,0.43650,6994.6216,0.36123,5914.0870,0.30880
7,*000J18,13822.4131,0.69253,13162.3039,0.66050,12564.6403,0.63150,12673.0441,0.63676,11509.6610,0.58031,10664.4822,0.53930,11259.8788,0.56819,9895.7632,0.50200,8894.3675,0.45341,9751.4996,0.49500,8201.6960,0.41980,7101.1706,0.36640,8545.8678,0.43650,6994.6216,0.36123,5914.0870,0.30880
8,*00N217,116771.7174,0.66277,113151.2630,0.64230,109530.8086,0.62183,103057.4999,0.58523,97995.5846,0.55661,92740.8851,0.52690,92509.1902,0.52559,85400.9311,0.48540,78310.3588,0.44531,78563.2777,0.44674,70561.8435,0.40150,62567.4841,0.35630,68881.6131,0.39200,58994.7836,0.33610,49037.2075,0.27980
9,*00N219,116771.7174,0.66277,113151.2630,0.64230,109530.8086,0.62183,103057.4999,0.58523,97995.5846,0.55661,92740.8851,0.52690,92509.1902,0.52559,85400.9311,0.48540,78310.3588,0.44531,78563.2777,0.44674,70561.8435,0.40150,62567.4841,0.35630,68881.6131,0.39200,58994.7836,0.33610,49037.2075,0.27980


#### Nodes

In [183]:
node_1 = []
node_2 = []
node_3 = []
node_4 = []
node_5 = []
node_6 = []
node_7 = []

'Loop through each row in the dataframe and assign node 1 (The vehicles make)'
for index, row in df_vehicles.iterrows():
    node_1.append(row['vcMake'])
df_vehicles['node_1'] = node_1

 
'Assign node 2 (The vehicle class)'
for index, row in df_vehicles.iterrows():
    if row['node_1'] == 'TOYOTA':
        if ('YARIS' in row['vcModel'] or
            row['vcModel'] == 'PRIUS-C' or
            row['vcModel'] == 'COROLLA' or
            row['vcModel'] == 'RUKUS' or
            row['vcModel'] == 'PRIUS' or
            row['vcModel'] == 'CAMRY' or
            row['vcModel'] == 'AURION' or
            row['vcModel'] == '86'):

            node_2.append('CAR')

        elif (row['vcModel'] == 'RAV4' or
            row['vcModel'] == 'C-HR' or
            row['vcModel'] == 'KLUGER' or
            row['vcModel'] == 'FORTUNER' or  
            row['vcModel'] == 'PRIUS V' or
            (row['vcModel'] == 'LANDCRUISER PRADO') or
            (row['vcModel'] == 'LANDCRUISER 200') or 
            row['vcModel'] == 'FJ CRUISER'):

            node_2.append('SUV')

        elif (row['vcModel'] == 'HILUX' or
            row['vcModel'] == 'TARAGO' or
            row['vcModel'] == 'HIACE' or
            row['vcModel'] == 'LANDCRUISER 70 SERIES'):

            node_2.append('LCV')

        else:
            node_2.append('UNKNOWN')
    else:
        node_2.append('UNKNOWN')
df_vehicles['node_2'] = node_2

'Assign node 3 (the vehicle model group)'
for index, row in df_vehicles.iterrows():
    if row['node_1'] != 'TOYOTA':
        node_3.append(row['vcModel'])
    else:
        if 'YARIS' in row['vcModel']  or row['vcModel'] == 'PRIUS-C':
            node_3.append('YARIS/PRIUS-C')
        elif row['vcModel'] == 'COROLLA' or row['vcModel'] == 'RUKUS':
            node_3.append('COROLLA/RUKUS')
        elif row['vcModel'] == 'CAMRY' or row['vcModel'] == 'AURION':
            node_3.append('CAMRY/AURION')
        elif row['vcModel'] == 'RAV4' or row['vcModel'] == 'C-HR':
            node_3.append('RAV4/C-HR')
        elif row['vcModel'] == 'KLUGER' or row['vcModel'] == 'FORTUNER' or row['vcModel'] == 'PRIUS V':
            node_3.append('KLUGER/FORTUNER/PRIUS V')
        elif (row['vcModel'] == 'LANDCRUISER PRADO'):
            node_3.append('PRADO')
        elif row['vcModel'] == 'FJ CRUISER' or (row['vcModel'] == 'LANDCRUISER 200'):
            node_3.append('LC 200/ FJ CRUISER')
        elif (row['vcModel'] == 'LANDCRUISER 70 SERIES'):
            node_3.append('LC 70')
        else:
            node_3.append(row['vcModel'])
df_vehicles['node_3'] = node_3

'Assign node 4 (this node level differs across vehicles, for some it is variant group for others it is cylinders or even engine size)'
for index, row in df_vehicles.iterrows():
    
    if (row['vcMake'] != 'TOYOTA'):
        node_4.append(row['vcVariant_1'])
    else:
        
        if (row['node_3'] == 'YARIS/PRIUS-C'):
            if 'ASCENT' in row['vcVariant'] or row['vcVariant'] == 'YR' or row['vcModel'] == 'PRIUS-C':
                node_4.append('ASCENT')
            else:
                node_4.append('OTHER')

        elif (row['node_3'] == 'COROLLA/RUKUS'):
            if (row['vcVariant'] == 'HYBRID' or 
                ('ASCENT' in row['vcVariant'] and 'ASCENT SPORT' not in row['vcVariant'])):
                node_4.append('ASCENT')
            elif 'RUKUS' in row['vcModel']:
                node_4.append('RUKUS')
            elif ('SPORT' in row['vcVariant'] or
                  'SX' in row['vcVariant'] or
                  'ZR' in row['vcVariant'] or
                  'LEVIN SECA' in row['vcVariant'] or
                  'ULTIMA' in row['vcVariant'] or 
                  'EDGE' in row['vcVariant'] or
                  'RZ' in row['vcVariant'] or
                  'CONQUEST' in row['vcVariant']):
                node_4.append('OTHER')
            else:
                node_4.append('UNKNOWN')

        elif (row['node_3'] == 'PRIUS'):
            node_4.append(row['vcEngineSize'])

        elif (row['node_3'] == 'CAMRY/AURION'):
            node_4.append(row['cCylinders'].strip())

        elif (row['node_3'] == '86' or row['node_3'] == 'BRZ' or row['node_3'] == 'MX-5'):
            row['node_3'] = '86'
            node_4.append(row['vcModel'])

        elif (row['node_3'] == 'RAV4/C-HR'):
            node_4.append(row['cCylinders'])

        elif (row['node_3'] == 'KLUGER/FORTUNER/PRIUS V'):
            node_4.append(row['clean_vcDriveType'])

        elif (row['node_3'] == 'PRADO'):
            if ('GX' in row['vcVariant'] or 
               'GXL' in row['vcVariant'] or
               'ADVANTAGE' in row['vcVariant'] or
               'ALTITUDE' in row['vcVariant'] or
               'PILBARA' in row['vcVariant']):
                node_4.append('STD')
            elif ('VX' in row['vcVariant'] or
                 'KAKADU' in row['vcVariant'] or
                 'ZR' in row['vcVariant'] or
                 'SX' in row['vcVariant']):
                node_4.append('LUX')
            else:
                node_4.append('UNKNOWN')

        elif (row['node_3'] == 'LC 200/ FJ CRUISER'):
            if ('LANDCRUISER 200' in row['vcModel']):
                node_4.append('LC 200')
            elif ('FJ CRUISER' in row['vcModel']):
                node_4.append('FJ CRUISER')
            else:
                node_4.append('UNKNOWN')

        elif (row['node_3'] == 'HILUX'):
            node_4.append(row['cCylinders'].strip())

        elif (row['node_3'] == 'LC 70'):
            if ('WAGON' in row['vcBodyStyle']):
                node_4.append('4D WAGON')
            elif ('CHAS' in row['vcBodyStyle']):
                node_4.append('CAB CHAS')
            elif ('TROOP' in row['vcBodyStyle']):
                node_4.append('TROOPCARRIER')
            else:
                node_4.append('UNKNOWN')

        elif (row['node_3'] == 'TARAGO'):
            node_4.append(row['cCylinders'])

        elif (row['node_3'] == 'HIACE'):
            if ('VAN' in row['vcBodyStyle']):
                node_4.append('VAN')   
            elif ('BUS' in row['vcBodyStyle']):
                node_4.append('BUS')
            else:
                node_4.append('UNKNOWN')

        else:
            node_4.append('UNKNOWN')
            
df_vehicles['node_4'] = node_4


'Assign node 5 ##############################################################################################'   
for index, row in df_vehicles.iterrows():
    
    if (row['vcMake'] != 'TOYOTA'):
            node_5.append(row['vcVariant_2'])
    else:
    
        if (row['node_3'] == 'YARIS/PRIUS-C'):
            if (row['node_4'] == 'OTHER'):
                if ('SX' in row['vcVariant'] or 'YRS' in row['vcVariant']):
                    node_5.append('SX')
                elif('ZR' in row['vcVariant'] or 'YRX' in row['vcVariant']):
                    node_5.append('ZR')
                else:
                    node_5.append('UNKNOWN')
            else:
                if (row['vcModel'] == 'PRIUS-C' and 'i-TECH' in row['vcVariant']):
                    node_5.append('i-TECH HYBRID')
                elif (row['vcModel'] == 'PRIUS-C' and 'i-TECH' not in row['vcVariant']):
                    node_5.append('HYBRID')
                else: 
                    node_5.append('ASCENT')

        elif (row['node_3'] == 'COROLLA/RUKUS'):
            node_5.append(row['clean_vcFuelType'])

        elif row['node_3'] == 'PRIUS':
            node_5.append('UNKNOWN')

        elif row['node_3'] == 'CAMRY/AURION':
            if (row['node_4'] == '4'):  
                node_5.append(row['clean_vcFuelType'])
            else:
                if row['vcModel'] == 'CAMRY':
                    node_5.append('AT-X')
                elif (row['vcModel'] == 'AURION' and
                     ('AT-X' in row['vcVariant'] or 'TOURING SE' in row['vcVariant'])):
                    node_5.append('AT-X')
                elif (row['vcModel'] == 'AURION' and
                      'PRESSARA' in row['vcVariant']):
                    node_5.append('PRESSARA')
                elif (row['vcModel'] == 'AURION' and
                      'PRODIGY' in row['vcVariant']):
                    node_5.append('PRODIGY')
                elif (row['vcModel'] == 'AURION' and
                     ('SPORT' in row['vcVariant'] or 'WHITE LIMITED' in row['vcVariant'])):
                    node_5.append('SPORT')
                else:
                    node_5.append('UNKNOWN')

        elif row['node_3'] == '86':
            node_5.append('UNKNOWN')

        elif row['node_3'] == 'RAV4/C-HR':
            if (row['node_4'] == '4'):
                node_5.append(row['clean_vcFuelType'])
            else:
                node_5.append('UNKNOWN')

        elif row['node_3'] == 'KLUGER/FORTUNER/PRIUS V':
            if (row['node_4']) == '2WD':
                if ('GRANDE' in row['vcVariant'] or 'i-TECH HYBRID' in row['vcVariant']):
                    node_5.append('GRANDE')
                elif (row['vcVariant_1'] == 'GXL' or 
                      row['vcVariant_1'] == 'CVX' or
                      row['vcVariant_1'] == 'KX-S'):
                    node_5.append('GXL')
                elif (row['vcVariant_1'] == 'GX' or 
                      row['vcVariant_1'] == 'CV' or
                      'KX-R' in row['vcVariant'] or 
                      'ALTITUDE' in row['vcVariant'] or 
                      'FORTUNER' in row['vcModel']):
                    node_5.append('GX')
                else:
                    node_5.append('UNKNOWN')
            elif (row['node_4'] == '4WD'):
                if ('GRANDE' in row['vcVariant']):
                    node_5.append('GRANDE')
                elif (row['vcVariant_1'] == 'GXL' or 
                      row['vcVariant_1'] == 'CVX' or
                      row['vcVariant_1'] == 'KX-S'):
                    node_5.append('GXL')
                elif (row['vcVariant_1'] == 'GX' or 
                      row['vcVariant_1'] == 'CV' or
                      'KX-R' in row['vcVariant'] or 
                      'ALTITUDE' in row['vcVariant'] or 
                      'FORTUNER' in row['vcModel']):
                    node_5.append('GX')
                else:
                    node_5.append('UNKNOWN')
            else:
                node_5.append('UNKNOWN')

        elif row['node_3'] == 'PRADO':
            if (row['node_4']) == 'LUX':
                if ('2D' in row['vcBodyStyle']):
                    node_5.append('2D WAGON')
                elif ('4D' in row['vcBodyStyle']) :
                    node_5.append('4D WAGON')
                else:
                    node_5.append('UNKNOWN')
            elif (row['node_4'] == 'STD'):
                node_5.append('UNKNOWN')
            else:
                node_5.append('UNKNOWN')


        elif row['node_3'] == 'LC 200/ FJ CRUISER':
            if (row['node_4'] == 'LC 200'):
                node_5.append(row['clean_vcFuelType'])
            else:
                node_5.append('UNKNOWN')

        elif row['node_3'] == 'HILUX':
            if (row['node_4'] == '4'):
                if (('SR' in row['vcVariant']) and ('SR5' not in row['vcVariant'])):
                    node_5.append('SR')
                elif ('SR5' in row['vcVariant'] or
                      'TRD' in row['vcVariant'] or
                      'RUGGED' in row['vcVariant'] or
                      'ROGUE' in row['vcVariant']):
                    node_5.append('SR5')
                elif ('WORKMATE' in row['vcVariant']):
                    node_5.append('WORKMATE')

            elif (row['node_4'] == '6') :
                if (('SR' in row['vcVariant']) and ('SR5' not in row['vcVariant'])):
                    node_5.append('SR')
                elif ('SR5' in row['vcVariant'] or
                      'TRD' in row['vcVariant'] or
                      'RUGGED' in row['vcVariant'] or
                      'ROGUE' in row['vcVariant']):
                    node_5.append('SR5')
                else:
                    node_5.append('UNKNOWN')
            else:
                node_5.append('UNKNOWN')

        elif row['node_3'] == 'LC 70':
            if (row['node_4'] == '4D WAGON'):
                if ('GXL' in row['vcVariant']):
                    node_5.append('GXL')
                elif (('WORKMATE' in row['vcVariant']) and ('SEAT' not in row['vcVariant'])):
                    node_5.append('WORKMATE')
                else:
                    node_5.append('UNKNOWN')
            elif (row['node_4'] == 'CAB CHAS'):
                if (('GX' in row['vcVariant']) and ('GXL' not in row['vcVariant'])):
                    node_5.append('GX')
                elif ('GXL' in row['vcVariant']):
                    node_5.append('GXL')
                elif (('WORKMATE' in row['vcVariant']) and ('SEAT' not in row['vcVariant'])):
                    node_5.append('WORKMATE')
                else:
                    node_5.append('UNKNOWN')
            elif (row['node_4'] == 'TROOPCARRIER'):
                if ('GXL' in row['vcVariant']):
                    node_5.append('GXL')
                elif (('WORKMATE' in row['vcVariant']) and ('11 SEAT' in row['vcVariant'])):
                    node_5.append('WORKMATE 11')
                elif (('WORKMATE' in row['vcVariant']) and ('SEAT' in row['vcVariant']) and ('11 SEAT' not in row['vcVariant'])):
                    node_5.append('WORKMATE 2')
                else:
                    node_5.append('UNKNOWN')
            else:
                    node_5.append('UNKNOWN')

        elif row['node_3'] == 'TARAGO':
            if (row['node_4'] == '4'):
                if ('GLi' in row['vcVariant']):
                    node_5.append('GLi')
                elif ('GLX' in row['vcVariant']):
                    node_5.append('GLX')
                else:
                    node_5.append('UNKNOWN')
            else:
                    node_5.append('UNKNOWN')

        elif row['node_3'] == 'HIACE':
            if (row['node_4'] == 'BUS'):
                node_5.append(row['clean_vcTransmissionType'].strip())
            elif (row['node_4'] == 'VAN'):
                if ('LWB' in row['vcVariant']):
                    node_5.append('LWB')
                elif ('SLWB' in row['vcVariant']):
                    node_5.append('SLWB')
                else:
                    node_5.append('UNKNOWN')
            else:
                    node_5.append('UNKNOWN')
        else:
                    node_5.append('UNKNOWN')
               
df_vehicles['node_5'] = node_5    
                

'Assign node 6 ##############################################################################################' 
for index, row in df_vehicles.iterrows():
    if (row['node_3'] == 'YARIS/PRIUS-C'):
        if (row['node_4'] == 'ASCENT'):
            if (row['node_5'] == 'ASCENT'):
                node_6.append(row['clean_vcTransmissionType'])
            else:
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'COROLLA/RUKUS'):
        if (row['node_4'] == 'ASCENT'):
            if (row['node_5'] == 'PETROL'):
                node_6.append(row['vcBodyStyle'].strip())
            else:
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'PRIUS'):
        node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'CAMRY/AURION'):
        if (row['node_4'] == '4'):
            if (row['node_5'] == 'HYBRID'):
                if ('HL' in row['vcVariant'] or 'SL' in row['vcVariant'] or 'LUX' in row['vcVariant']):
                    node_6.append('LUX')
                else:
                    node_6.append('STD')
            elif (row['node_5'] == 'PETROL'):
                if ('GRANDE' in row['vcVariant'] or 'SL' in row['vcVariant'] or 'AZURA' in row['vcVariant']):
                    node_6.append('SL')
                elif ('SX' in row['vcVariant'] or 'TOURING' in row['vcVariant'] or 'SPORTIVO' in row['vcVariant']):
                    node_6.append('SX')
                elif ('ATARA R' in row['vcVariant'] or 'ATARA S' in row['vcVariant'] or 'ATEVA' in row['vcVariant'] or 'CONQUEST' in row['vcVariant'] or 'ASCENT SPORT' in row['vcVariant']):
                    node_6.append('SPORT')
                elif ( 'ALTISE' in row['vcVariant'] or 'RZ' in row['vcVariant'] or 'ASCENT' in row['vcVariant']):
                    node_6.append('ASCENT')
                else: 
                    node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == '86'):
        node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'RAV4/C-HR'):
        if (row['node_4'] == '4'):
            if (row['node_5'] == 'PETROL'):
                node_6.append(row['vcModel'])
            else:
                node_6.append('UNKNOWN')
        else:
                node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'KLUGER/FORTUNER/PRIUS V'):
        if (row['node_4'] == '2WD'):
            if (row['node_5'] == 'GRANDE'):
                if ('GRANDE' in row['vcVariant']):
                    node_6.append('GRANDE')
                elif ('i-TECH' in row['vcVariant']):
                    node_6.append('i-TECH HYBRID')
                else:
                    node_6.append('UNKNOWN')
            elif (row['node_5'] == 'GX'):
                node_6.append('UNKNOWN')
            elif (row['node_5'] == 'GXL'):
                if ('CVX' in row['vcVariant'] or 'GXL' in row['vcVariant'] or 'KX-S' in row['vcVariant'] or (row['vcModel'] == 'PRIUS V' and 'HYBRID' in row['vcVariant'])):
                    node_6.append('GXL')
                else:
                    node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        
        elif (row['node_4'] == '4WD'):
            if (row['node_5'] == 'GRANDE'):
                node_6.append('UNKNOWN')
            elif (row['node_5'] == 'GX'):
                node_6.append(row['vcModel'])
            elif (row['node_5'] == 'GXL'):
                node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'PRADO'):
        node_6.append('UNKNOWN')
        
    elif (row['node_3'] == 'LC 200/ FJ CRUISER'):
        if (row['node_4'] == 'LC 200'):
            if (row['node_5'] == 'PETROL'):
                if ('GX' in row['vcVariant'] or 'ALTITUDE' in row['vcVariant'] or 'ANNIVERSARY' in row['vcVariant']):
                    node_6.append('STD')
                elif ('SAHARA' in row['vcVariant'] or 'VX' in row['vcVariant'] or 'KAKADU' in row['vcVariant']):
                    node_6.append('LUX')
                else:
                    node_6.append('UNKNOWN')
            elif (row['node_5'] == 'DIESEL'):
                if (('GX' in row['vcVariant'] and 'GXL' not in row['vcVariant'])):
                    node_6.append('GX')
                elif ('GXL' in row['vcVariant'] or 'ALTITUDE' in row['vcVariant'] or 'ANNIVERSARY' in row['vcVariant']):
                    node_6.append('GXL')
                elif ('VX' in row['vcVariant'] or 'KAKADU' in row['vcVariant']):
                    node_6.append('VX')
                elif ('SAHARA ' in row['vcVariant']):
                    node_6.append('SAHARA')
                else:
                    node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        else:
                    node_6.append('UNKNOWN')  

    elif (row['node_3'] == 'HILUX'):
        if (row['node_4'] == '4'):
            if (row['node_5'] == 'SR'):
                if (row['vcBodyStyle'] == 'C/CHAS'):
                    node_6.append('CC')
                elif ('C/CHAS' in row['vcBodyStyle'] and ('DUAL' in row['vcBodyStyle'] or 'DOUBLE' in row['vcBodyStyle'])):
                    node_6.append('DUAL CC')
                elif (('P/UP' in row['vcBodyStyle'] or 'UTILITY' in row['vcBodyStyle']) and ('DUAL' in row['vcBodyStyle'] or 'DOUBLE' in row['vcBodyStyle'])):
                    node_6.append('DUAL UTE')
                elif ('C/CHAS' in row['vcBodyStyle'] and 'X' in row['vcBodyStyle']):
                    node_6.append('X CAB CC')
                elif (('P/UP' in row['vcBodyStyle'] or 'UTILITY' in row['vcBodyStyle']) and 'X' in row['vcBodyStyle']):
                    node_6.append('X CAB UTE')
                elif ('P/UP' in row['vcBodyStyle'] or 'UTILITY' in row['vcBodyStyle']):
                    node_6.append('UTE')
                else:
                    node_6.append('UNKNOWN')
            
            elif (row['node_5'] == 'SR5'):
                node_6.append('UNKNOWN')
            elif (row['node_5'] == 'WORKMATE'):
                if (row['vcBodyStyle'] == 'C/CHAS'):
                    node_6.append('CC')
                elif (('P/UP' in row['vcBodyStyle'] or 'UTILITY' in row['vcBodyStyle']) and ('DUAL' in row['vcBodyStyle'] or 'DOUBLE' in row['vcBodyStyle'])):
                    node_6.append('DUAL UTE')
                else:
                    node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        
        elif (row['node_4'] == '6'):
            if (row['node_5'] == 'SR'):
                node_6.append('UNKNOWN')
            elif (row['node_5'] == 'SR5'):
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'LC 70'):
        node_6.append ('UNKNOWN')
        
    elif (row['node_3'] == 'TARAGO'):
        node_6.append ('UNKNOWN')
    
    elif (row['node_3'] == 'HIACE'):
        node_6.append ('UNKNOWN')
        
    else:
        node_6.append ('UNKNOWN')
        
df_vehicles['node_6'] = node_6



'Assign node 7 ##############################################################################################'

for index, row in df_vehicles.iterrows():
    if (row['node_3'] == 'YARIS/PRIUS-C'):
        if (row['node_4'] == 'ASCENT'):
            if (row['node_5'] == 'ASCENT'):
                if (row['node_6'] == 'AUTO'):
                    if ('3D' in row['vcBodyStyle']):
                        node_7.append('3D HATCH')
                    elif ('5D' in row['vcBodyStyle']):
                        node_7.append('5D HATCH')
                    else:
                        node_7.append('UNKNOWN')
                elif (row['node_6'] == 'MANUAL'):
                    if ('3D' in row['vcBodyStyle']):
                        node_7.append('3D HATCH')
                    elif ('5D' in row['vcBodyStyle']):
                        node_7.append('5D HATCH')
                    else:
                        node_7.append('UNKNOWN')
                else:
                    node_7.append('UNKNOWN')
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'COROLLA/RUKUS'):
        if (row['node_4'] == 'ASCENT'):
            if (row['node_5'] == 'PETROL'):
                if (row['node_6'] == '4D SEDAN'):
                    node_7.append(row['clean_vcTransmissionType'])
                elif (row['node_6'] == '5D HATCH'):
                    node_7.append(row['clean_vcTransmissionType'])
                else:
                    node_7.append('UNKNOWN')
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'PRIUS'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'CAMRY/AURION'):
        if(row['node_4'] == '4'):
            if(row['node_5'] == 'HYBRID'):
                if(row['node_6'] == 'LUX'):
                    if ('HL' in row['vcVariant']):
                        node_7.append('HL')
                    elif ('SL' in row['vcVariant']):
                        node_7.append('SL')
                    elif ('LUX' in row['vcVariant']):
                        node_7.append('LUX')
                    else:
                        node_7.append('UNKNOWN')
                else:
                    if ('ATARA R' in row['vcVariant'] or 'ATARA S' in row['vcVariant'] or 'ATEVA' in row['vcVariant'] or 'CONQUEST' in row['vcVariant'] or 'ASCENT SPORT' in row['vcVariant']):
                        node_7.append('SPORT')
                    elif ( 'ALTISE' in row['vcVariant'] or 'RZ' in row['vcVariant'] or 'ASCENT' in row['vcVariant']):
                        node_7.append('ASCENT')
                    else:
                        node_7.append(row['vcVariant'])
            
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
                    
    elif (row['node_3'] == '86'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'RAV4/C-HR'):
        if(row['node_4'] == '4'):
            if(row['node_5'] == 'PETROL'):
                if(row['node_6'] == 'RAV4'):
                    node_7.append(row['clean_vcEngineSize'])
                else:
                    node_7.append('UNKNOWN')
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'KLUGER/FORTUNER/PRIUS V'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'PRADO'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'LC 200/ FJ CRUISER'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'HILUX'):
        if (row['node_4'] == '4'):
            if (row['node_5'] == 'SR'):
                if (row['node_6'] == 'DUAL CC'):
                    node_7.append(row['clean_vcEngineSize'])
                elif (row['node_6'] == 'DUAL UTE'):
                    node_7.append(row['clean_vcEngineSize'])
                else:
                    node_7.append('UNKNOWN')
            
            elif(row['node_5'] == 'WORKMATE'):
                if (row['node_6'] == 'CC'):
                    node_7.append(row['clean_vcFuelType'])
                elif (row['node_6'] == 'DUAL UTE'):
                    node_7.append(row['clean_vcFuelType'])
                else:
                    node_7.append('UNKNOWN')
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'LC 70'):
        node_7.append('UNKNOWN')
        
    elif (row['node_3'] == 'TARAGO'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'HIACE'):
        node_7.append('UNKNOWN')
    else:
        node_7.append('UNKNOWN')
df_vehicles['node_7'] = node_7    



In [184]:
df_vehicles.head()

,nvic,vcMake,vcModel,vcSeries,vcVariant,vcBodyStyle,vcTransmissionType,vcDriveType,vcFuelType,vcEngineSize,cCylinders,dtReleaseDate,node_1,node_2,node_3,make_model,nvic_family,nvic_year,make_model_nvic_family,clean_vcVariant,clean_vcTransmissionType,clean_vcFuelType,clean_vcBodyStyle,clean_vcDriveType,clean_vcEngineSize,vcVariant_1,vcVariant_2,node_4,node_5,node_6,node_7
0,*000F17,TOYOTA,YARIS,NCP130R MY17,ASCENT,5D HATCHBACK,4 SP AUTOMATIC,Front wheel drive,UNLEADED PETROL,1.3L,4,2017-03-01,TOYOTA,CAR,YARIS/PRIUS-C,TOYOTA_YARIS,*000F,17,TOYOTA_YARIS_*000F,ASCENT,AUTO,PETROL,HATCH,2WD,1.3L,ASCENT,,ASCENT,ASCENT,AUTO,5D HATCH
1,*000F18,TOYOTA,YARIS,NCP130R MY17,ASCENT,5D HATCHBACK,4 SP AUTOMATIC,Front wheel drive,UNLEADED PETROL,1.3L,4,2018-01-01,TOYOTA,CAR,YARIS/PRIUS-C,TOYOTA_YARIS,*000F,18,TOYOTA_YARIS_*000F,ASCENT,AUTO,PETROL,HATCH,2WD,1.3L,ASCENT,,ASCENT,ASCENT,AUTO,5D HATCH
2,*000G17,TOYOTA,YARIS,NCP130R MY17,ASCENT,5D HATCHBACK,5 SP MANUAL,Front wheel drive,UNLEADED PETROL,1.3L,4,2017-03-01,TOYOTA,CAR,YARIS/PRIUS-C,TOYOTA_YARIS,*000G,17,TOYOTA_YARIS_*000G,ASCENT,MANUAL,PETROL,HATCH,2WD,1.3L,ASCENT,,ASCENT,ASCENT,MANUAL,5D HATCH
3,*000G18,TOYOTA,YARIS,NCP130R MY17,ASCENT,5D HATCHBACK,5 SP MANUAL,Front wheel drive,UNLEADED PETROL,1.3L,4,2018-01-01,TOYOTA,CAR,YARIS/PRIUS-C,TOYOTA_YARIS,*000G,18,TOYOTA_YARIS_*000G,ASCENT,MANUAL,PETROL,HATCH,2WD,1.3L,ASCENT,,ASCENT,ASCENT,MANUAL,5D HATCH
4,*000H17,TOYOTA,YARIS,NCP131R MY17,SX,5D HATCHBACK,4 SP AUTOMATIC,Front wheel drive,UNLEADED PETROL,1.5L,4,2017-03-01,TOYOTA,CAR,YARIS/PRIUS-C,TOYOTA_YARIS,*000H,17,TOYOTA_YARIS_*000H,SX,AUTO,PETROL,HATCH,2WD,1.5L,SX,,OTHER,SX,UNKNOWN,UNKNOWN


In [101]:
node_key = []

for index, row in df_vehicles.iterrows():
    final_node = row['node_1'].strip() + ' ' + row['node_2'].strip() + ' ' + row['node_3'].strip() + ' ' + row['node_4'].strip() + ' ' + row['node_5'].strip() + ' ' + row['node_6'].strip() + ' ' + row['node_7'].strip() 
    temp = final_node.replace('UNKNOWN', '')
    node_key.append(temp.replace('  ', ' '))

df_vehicles['node_key'] = node_key

#### Join RV & Node tables

In [106]:
df_vehicles = pd.merge(df_vehicles, df_vehicles_rv_band, how='left', on='nvic')

# Joining GFV model output to vehicle dataset

### Reading GFV model output from excel

In [107]:
df_gfv_model_outputs = pd.read_csv('H:\\Credit Operations\\General\\Asset Risk - Data Science team\\Initiatives\\202102 GFV management tool\\Data\\gfv_model_output.csv')

In [108]:
df_gfvs = df_gfv_model_outputs[['cVehicleCode', 'vcMake', 'vcModel','age','kkm','rv','valuation']].copy()
df_gfvs['cVehicleCode'] = '*' + df_gfvs['cVehicleCode'].str.strip()

### Joining GFVs with vehicle dataset

In [109]:
# Data processing for gfv dataset
term_kkms = [[12, 10],
             [12, 20],
             [12, 30],
             [24, 20],
             [24, 40],
             [24, 60],
             [36, 30],
             [36, 60], 
             [36, 90], 
             [48, 40],
             [48, 80],
             [48, 120],
             [60, 50],
             [60, 100],
             [60, 150]]

term_kkm_count = 0
loop_count = 0
df_vehicles_final = None
df_vehicles_temp = None
for term_kkm in term_kkms:
    term_band = str(term_kkm[0]) + '_' + str(int(term_kkm[1]))
    df_vehicles_temp = df_gfvs[(df_gfvs['age'] == term_kkm[0]) & 
                              (df_gfvs['kkm'] == term_kkm[1])][['cVehicleCode', 'rv', 'valuation']]
    df_vehicles_temp.columns = ['nvic', 'RV_model_' + term_band, 'GFVAmount_' + term_band]

    if term_kkm_count == 0:
        df_vehicles_final = pd.merge(df_vehicles_uniques, df_vehicles_temp, how='left', on='nvic')
    else:
        df_vehicles_final = pd.merge(df_vehicles_final, df_vehicles_temp, how='left', on='nvic')
    #print(term_kkm[1])    
    #df_vehicles_final = df_vehicles_final.dropna()
    df_vehicles_final = df_vehicles_final.reset_index(drop=True)
    term_kkm_count += 1
    
    loop_count += 1
    #if loop_count >= 1:
       # break
df_vehicles_gfvs = df_vehicles_final

In [110]:
df_vehicles = pd.merge(df_vehicles, df_vehicles_gfvs, how='left', on='nvic')

# Flagging Vehicles as Modelled/Unmodelled

In [112]:
df_modelled_vehicles = pd.read_csv('H:\\Credit Operations\\General\\Asset Risk - Data Science team\\Initiatives\\202102 GFV management tool\\Data\\modelled_vehicles.csv')

In [113]:
modelled_vehicles = df_modelled_vehicles['modelled_vehicles'].tolist()
modelled_flag = []

for index, row in df_vehicles.iterrows():
    if row['make_model'].strip() in modelled_vehicles:
        modelled_flag.append("Y")
    else:
        modelled_flag.append("N")
df_vehicles['modelled_flag'] = modelled_flag

# Disposal Cleaning

In [159]:
for index, row in df_disposals.iterrows():
    if row['vcModel'] == 'LANDCRUISER':
        if 'PRADO' in row['vcVariant']:
            df_disposals.loc[index,'vcModel'] = 'LANDCRUISER PRADO'
            next
        elif ('200' in row['vcVariant'] or (row['vcSeries'][3:4]) == '2' and row['vcSeries'][4:5] == '0'):
            df_disposals.loc[index,'vcModel'] = 'LANDCRUISER 200'
            next
        elif '7' in row['vcSeries'][:4]:
            df_disposals.loc[index,'vcModel'] = 'LANDCRUISER 70 SERIES'
        else:
            next
    else:
        next

In [160]:
df_disposals = dp.vehicle_cleaning_variantkey(df_disposals)

In [162]:
node_1 = []
node_2 = []
node_3 = []
node_4 = []
node_5 = []
node_6 = []
node_7 = []

'Loop through each row in the dataframe and assign node 1 (The vehicles make)'
for index, row in df_disposals.iterrows():
    node_1.append(row['vcMake'])
df_disposals['node_1'] = node_1

 
'Assign node 2 (The vehicle class)'
for index, row in df_disposals.iterrows():
    if row['node_1'] == 'TOYOTA':
        if ('YARIS' in row['vcModel'] or
            row['vcModel'] == 'PRIUS-C' or
            row['vcModel'] == 'COROLLA' or
            row['vcModel'] == 'RUKUS' or
            row['vcModel'] == 'PRIUS' or
            row['vcModel'] == 'CAMRY' or
            row['vcModel'] == 'AURION' or
            row['vcModel'] == '86'):

            node_2.append('CAR')

        elif (row['vcModel'] == 'RAV4' or
            row['vcModel'] == 'C-HR' or
            row['vcModel'] == 'KLUGER' or
            row['vcModel'] == 'FORTUNER' or  
            row['vcModel'] == 'PRIUS V' or
            (row['vcModel'] == 'LANDCRUISER PRADO') or
            (row['vcModel'] == 'LANDCRUISER 200') or 
            row['vcModel'] == 'FJ CRUISER'):

            node_2.append('SUV')

        elif (row['vcModel'] == 'HILUX' or
            row['vcModel'] == 'TARAGO' or
            row['vcModel'] == 'HIACE' or
            row['vcModel'] == 'LANDCRUISER 70 SERIES'):

            node_2.append('LCV')

        else:
            node_2.append('UNKNOWN')
    else:
        node_2.append('UNKNOWN')
df_disposals['node_2'] = node_2

'Assign node 3 (the vehicle model group)'
for index, row in df_disposals.iterrows():
    if row['node_1'] != 'TOYOTA':
        node_3.append(row['vcModel'])
    else:
        if 'YARIS' in row['vcModel']  or row['vcModel'] == 'PRIUS-C':
            node_3.append('YARIS/PRIUS-C')
        elif row['vcModel'] == 'COROLLA' or row['vcModel'] == 'RUKUS':
            node_3.append('COROLLA/RUKUS')
        elif row['vcModel'] == 'CAMRY' or row['vcModel'] == 'AURION':
            node_3.append('CAMRY/AURION')
        elif row['vcModel'] == 'RAV4' or row['vcModel'] == 'C-HR':
            node_3.append('RAV4/C-HR')
        elif row['vcModel'] == 'KLUGER' or row['vcModel'] == 'FORTUNER' or row['vcModel'] == 'PRIUS V':
            node_3.append('KLUGER/FORTUNER/PRIUS V')
        elif (row['vcModel'] == 'LANDCRUISER PRADO'):
            node_3.append('PRADO')
        elif row['vcModel'] == 'FJ CRUISER' or (row['vcModel'] == 'LANDCRUISER 200'):
            node_3.append('LC 200/ FJ CRUISER')
        elif (row['vcModel'] == 'LANDCRUISER 70 SERIES'):
            node_3.append('LC 70')
        else:
            node_3.append(row['vcModel'])
df_disposals['node_3'] = node_3

'Assign node 4 (this node level differs across vehicles, for some it is variant group for others it is cylinders or even engine size)'
for index, row in df_disposals.iterrows():
    
    if (row['vcMake'] != 'TOYOTA'):
        node_4.append(row['vcVariant_1'])
    else:
        
        if (row['node_3'] == 'YARIS/PRIUS-C'):
            if 'ASCENT' in row['vcVariant'] or row['vcVariant'] == 'YR' or row['vcModel'] == 'PRIUS-C':
                node_4.append('ASCENT')
            else:
                node_4.append('OTHER')

        elif (row['node_3'] == 'COROLLA/RUKUS'):
            if (row['vcVariant'] == 'HYBRID' or 
                ('ASCENT' in row['vcVariant'] and 'ASCENT SPORT' not in row['vcVariant'])):
                node_4.append('ASCENT')
            elif 'RUKUS' in row['vcModel']:
                node_4.append('RUKUS')
            elif ('SPORT' in row['vcVariant'] or
                  'SX' in row['vcVariant'] or
                  'ZR' in row['vcVariant'] or
                  'LEVIN SECA' in row['vcVariant'] or
                  'ULTIMA' in row['vcVariant'] or 
                  'EDGE' in row['vcVariant'] or
                  'RZ' in row['vcVariant'] or
                  'CONQUEST' in row['vcVariant']):
                node_4.append('OTHER')
            else:
                node_4.append('UNKNOWN')

        elif (row['node_3'] == 'PRIUS'):
            node_4.append(row['vcEngineSize'])

        elif (row['node_3'] == 'CAMRY/AURION'):
            node_4.append(row['cCylinders'].strip())

        elif (row['node_3'] == '86' or row['node_3'] == 'BRZ' or row['node_3'] == 'MX-5'):
            row['node_3'] = '86'
            node_4.append(row['vcModel'])

        elif (row['node_3'] == 'RAV4/C-HR'):
            node_4.append(row['cCylinders'])

        elif (row['node_3'] == 'KLUGER/FORTUNER/PRIUS V'):
            node_4.append(row['clean_vcDriveType'])

        elif (row['node_3'] == 'PRADO'):
            if ('GX' in row['vcVariant'] or 
               'GXL' in row['vcVariant'] or
               'ADVANTAGE' in row['vcVariant'] or
               'ALTITUDE' in row['vcVariant'] or
               'PILBARA' in row['vcVariant']):
                node_4.append('STD')
            elif ('VX' in row['vcVariant'] or
                 'KAKADU' in row['vcVariant'] or
                 'ZR' in row['vcVariant'] or
                 'SX' in row['vcVariant']):
                node_4.append('LUX')
            else:
                node_4.append('UNKNOWN')

        elif (row['node_3'] == 'LC 200/ FJ CRUISER'):
            if ('LANDCRUISER 200' in row['vcModel']):
                node_4.append('LC 200')
            elif ('FJ CRUISER' in row['vcModel']):
                node_4.append('FJ CRUISER')
            else:
                node_4.append('UNKNOWN')

        elif (row['node_3'] == 'HILUX'):
            node_4.append(row['cCylinders'].strip())

        elif (row['node_3'] == 'LC 70'):
            if ('WAGON' in row['vcBodyStyle']):
                node_4.append('4D WAGON')
            elif ('CHAS' in row['vcBodyStyle']):
                node_4.append('CAB CHAS')
            elif ('TROOP' in row['vcBodyStyle']):
                node_4.append('TROOPCARRIER')
            else:
                node_4.append('UNKNOWN')

        elif (row['node_3'] == 'TARAGO'):
            node_4.append(row['cCylinders'])

        elif (row['node_3'] == 'HIACE'):
            if ('VAN' in row['vcBodyStyle']):
                node_4.append('VAN')   
            elif ('BUS' in row['vcBodyStyle']):
                node_4.append('BUS')
            else:
                node_4.append('UNKNOWN')

        else:
            node_4.append('UNKNOWN')
            
df_disposals['node_4'] = node_4


'Assign node 5 ##############################################################################################'   
for index, row in df_disposals.iterrows():
    
    if (row['vcMake'] != 'TOYOTA'):
            node_5.append(row['vcVariant_2'])
    else:
    
        if (row['node_3'] == 'YARIS/PRIUS-C'):
            if (row['node_4'] == 'OTHER'):
                if ('SX' in row['vcVariant'] or 'YRS' in row['vcVariant']):
                    node_5.append('SX')
                elif('ZR' in row['vcVariant'] or 'YRX' in row['vcVariant']):
                    node_5.append('ZR')
                else:
                    node_5.append('UNKNOWN')
            else:
                if (row['vcModel'] == 'PRIUS-C' and 'i-TECH' in row['vcVariant']):
                    node_5.append('i-TECH HYBRID')
                elif (row['vcModel'] == 'PRIUS-C' and 'i-TECH' not in row['vcVariant']):
                    node_5.append('HYBRID')
                else: 
                    node_5.append('ASCENT')

        elif (row['node_3'] == 'COROLLA/RUKUS'):
            node_5.append(row['clean_vcFuelType'])

        elif row['node_3'] == 'PRIUS':
            node_5.append('UNKNOWN')

        elif row['node_3'] == 'CAMRY/AURION':
            if (row['node_4'] == '4'):  
                node_5.append(row['clean_vcFuelType'])
            else:
                if row['vcModel'] == 'CAMRY':
                    node_5.append('AT-X')
                elif (row['vcModel'] == 'AURION' and
                     ('AT-X' in row['vcVariant'] or 'TOURING SE' in row['vcVariant'])):
                    node_5.append('AT-X')
                elif (row['vcModel'] == 'AURION' and
                      'PRESSARA' in row['vcVariant']):
                    node_5.append('PRESSARA')
                elif (row['vcModel'] == 'AURION' and
                      'PRODIGY' in row['vcVariant']):
                    node_5.append('PRODIGY')
                elif (row['vcModel'] == 'AURION' and
                     ('SPORT' in row['vcVariant'] or 'WHITE LIMITED' in row['vcVariant'])):
                    node_5.append('SPORT')
                else:
                    node_5.append('UNKNOWN')

        elif row['node_3'] == '86':
            node_5.append('UNKNOWN')

        elif row['node_3'] == 'RAV4/C-HR':
            if (row['node_4'] == '4'):
                node_5.append(row['clean_vcFuelType'])
            else:
                node_5.append('UNKNOWN')

        elif row['node_3'] == 'KLUGER/FORTUNER/PRIUS V':
            if (row['node_4']) == '2WD':
                if ('GRANDE' in row['vcVariant'] or 'i-TECH HYBRID' in row['vcVariant']):
                    node_5.append('GRANDE')
                elif (row['vcVariant_1'] == 'GXL' or 
                      row['vcVariant_1'] == 'CVX' or
                      row['vcVariant_1'] == 'KX-S'):
                    node_5.append('GXL')
                elif (row['vcVariant_1'] == 'GX' or 
                      row['vcVariant_1'] == 'CV' or
                      'KX-R' in row['vcVariant'] or 
                      'ALTITUDE' in row['vcVariant'] or 
                      'FORTUNER' in row['vcModel']):
                    node_5.append('GX')
                else:
                    node_5.append('UNKNOWN')
            elif (row['node_4'] == '4WD'):
                if ('GRANDE' in row['vcVariant']):
                    node_5.append('GRANDE')
                elif (row['vcVariant_1'] == 'GXL' or 
                      row['vcVariant_1'] == 'CVX' or
                      row['vcVariant_1'] == 'KX-S'):
                    node_5.append('GXL')
                elif (row['vcVariant_1'] == 'GX' or 
                      row['vcVariant_1'] == 'CV' or
                      'KX-R' in row['vcVariant'] or 
                      'ALTITUDE' in row['vcVariant'] or 
                      'FORTUNER' in row['vcModel']):
                    node_5.append('GX')
                else:
                    node_5.append('UNKNOWN')
            else:
                node_5.append('UNKNOWN')

        elif row['node_3'] == 'PRADO':
            if (row['node_4']) == 'LUX':
                if ('2D' in row['vcBodyStyle']):
                    node_5.append('2D WAGON')
                elif ('4D' in row['vcBodyStyle']) :
                    node_5.append('4D WAGON')
                else:
                    node_5.append('UNKNOWN')
            elif (row['node_4'] == 'STD'):
                node_5.append('UNKNOWN')
            else:
                node_5.append('UNKNOWN')


        elif row['node_3'] == 'LC 200/ FJ CRUISER':
            if (row['node_4'] == 'LC 200'):
                node_5.append(row['clean_vcFuelType'])
            else:
                node_5.append('UNKNOWN')

        elif row['node_3'] == 'HILUX':
            if (row['node_4'] == '4'):
                if (('SR' in row['vcVariant']) and ('SR5' not in row['vcVariant'])):
                    node_5.append('SR')
                elif ('SR5' in row['vcVariant'] or
                      'TRD' in row['vcVariant'] or
                      'RUGGED' in row['vcVariant'] or
                      'ROGUE' in row['vcVariant']):
                    node_5.append('SR5')
                elif ('WORKMATE' in row['vcVariant']):
                    node_5.append('WORKMATE')

            elif (row['node_4'] == '6') :
                if (('SR' in row['vcVariant']) and ('SR5' not in row['vcVariant'])):
                    node_5.append('SR')
                elif ('SR5' in row['vcVariant'] or
                      'TRD' in row['vcVariant'] or
                      'RUGGED' in row['vcVariant'] or
                      'ROGUE' in row['vcVariant']):
                    node_5.append('SR5')
                else:
                    node_5.append('UNKNOWN')
            else:
                node_5.append('UNKNOWN')

        elif row['node_3'] == 'LC 70':
            if (row['node_4'] == '4D WAGON'):
                if ('GXL' in row['vcVariant']):
                    node_5.append('GXL')
                elif (('WORKMATE' in row['vcVariant']) and ('SEAT' not in row['vcVariant'])):
                    node_5.append('WORKMATE')
                else:
                    node_5.append('UNKNOWN')
            elif (row['node_4'] == 'CAB CHAS'):
                if (('GX' in row['vcVariant']) and ('GXL' not in row['vcVariant'])):
                    node_5.append('GX')
                elif ('GXL' in row['vcVariant']):
                    node_5.append('GXL')
                elif (('WORKMATE' in row['vcVariant']) and ('SEAT' not in row['vcVariant'])):
                    node_5.append('WORKMATE')
                else:
                    node_5.append('UNKNOWN')
            elif (row['node_4'] == 'TROOPCARRIER'):
                if ('GXL' in row['vcVariant']):
                    node_5.append('GXL')
                elif (('WORKMATE' in row['vcVariant']) and ('11 SEAT' in row['vcVariant'])):
                    node_5.append('WORKMATE 11')
                elif (('WORKMATE' in row['vcVariant']) and ('SEAT' in row['vcVariant']) and ('11 SEAT' not in row['vcVariant'])):
                    node_5.append('WORKMATE 2')
                else:
                    node_5.append('UNKNOWN')
            else:
                    node_5.append('UNKNOWN')

        elif row['node_3'] == 'TARAGO':
            if (row['node_4'] == '4'):
                if ('GLi' in row['vcVariant']):
                    node_5.append('GLi')
                elif ('GLX' in row['vcVariant']):
                    node_5.append('GLX')
                else:
                    node_5.append('UNKNOWN')
            else:
                    node_5.append('UNKNOWN')

        elif row['node_3'] == 'HIACE':
            if (row['node_4'] == 'BUS'):
                node_5.append(row['clean_vcTransmissionType'].strip())
            elif (row['node_4'] == 'VAN'):
                if ('LWB' in row['vcVariant']):
                    node_5.append('LWB')
                elif ('SLWB' in row['vcVariant']):
                    node_5.append('SLWB')
                else:
                    node_5.append('UNKNOWN')
            else:
                    node_5.append('UNKNOWN')
        else:
                    node_5.append('UNKNOWN')
               
df_disposals['node_5'] = node_5    
                

'Assign node 6 ##############################################################################################' 
for index, row in df_disposals.iterrows():
    if (row['node_3'] == 'YARIS/PRIUS-C'):
        if (row['node_4'] == 'ASCENT'):
            if (row['node_5'] == 'ASCENT'):
                node_6.append(row['clean_vcTransmissionType'])
            else:
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'COROLLA/RUKUS'):
        if (row['node_4'] == 'ASCENT'):
            if (row['node_5'] == 'PETROL'):
                node_6.append(row['vcBodyStyle'].strip())
            else:
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'PRIUS'):
        node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'CAMRY/AURION'):
        if (row['node_4'] == '4'):
            if (row['node_5'] == 'HYBRID'):
                if ('HL' in row['vcVariant'] or 'SL' in row['vcVariant'] or 'LUX' in row['vcVariant']):
                    node_6.append('LUX')
                else:
                    node_6.append('STD')
            elif (row['node_5'] == 'PETROL'):
                if ('GRANDE' in row['vcVariant'] or 'SL' in row['vcVariant'] or 'AZURA' in row['vcVariant']):
                    node_6.append('SL')
                elif ('SX' in row['vcVariant'] or 'TOURING' in row['vcVariant'] or 'SPORTIVO' in row['vcVariant']):
                    node_6.append('SX')
                elif ('ATARA R' in row['vcVariant'] or 'ATARA S' in row['vcVariant'] or 'ATEVA' in row['vcVariant'] or 'CONQUEST' in row['vcVariant'] or 'ASCENT SPORT' in row['vcVariant']):
                    node_6.append('SPORT')
                elif ( 'ALTISE' in row['vcVariant'] or 'RZ' in row['vcVariant'] or 'ASCENT' in row['vcVariant']):
                    node_6.append('ASCENT')
                else: 
                    node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == '86'):
        node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'RAV4/C-HR'):
        if (row['node_4'] == '4'):
            if (row['node_5'] == 'PETROL'):
                node_6.append(row['vcModel'])
            else:
                node_6.append('UNKNOWN')
        else:
                node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'KLUGER/FORTUNER/PRIUS V'):
        if (row['node_4'] == '2WD'):
            if (row['node_5'] == 'GRANDE'):
                if ('GRANDE' in row['vcVariant']):
                    node_6.append('GRANDE')
                elif ('i-TECH' in row['vcVariant']):
                    node_6.append('i-TECH HYBRID')
                else:
                    node_6.append('UNKNOWN')
            elif (row['node_5'] == 'GX'):
                node_6.append('UNKNOWN')
            elif (row['node_5'] == 'GXL'):
                if ('CVX' in row['vcVariant'] or 'GXL' in row['vcVariant'] or 'KX-S' in row['vcVariant'] or (row['vcModel'] == 'PRIUS V' and 'HYBRID' in row['vcVariant'])):
                    node_6.append('GXL')
                else:
                    node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        
        elif (row['node_4'] == '4WD'):
            if (row['node_5'] == 'GRANDE'):
                node_6.append('UNKNOWN')
            elif (row['node_5'] == 'GX'):
                node_6.append(row['vcModel'])
            elif (row['node_5'] == 'GXL'):
                node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'PRADO'):
        node_6.append('UNKNOWN')
        
    elif (row['node_3'] == 'LC 200/ FJ CRUISER'):
        if (row['node_4'] == 'LC 200'):
            if (row['node_5'] == 'PETROL'):
                if ('GX' in row['vcVariant'] or 'ALTITUDE' in row['vcVariant'] or 'ANNIVERSARY' in row['vcVariant']):
                    node_6.append('STD')
                elif ('SAHARA' in row['vcVariant'] or 'VX' in row['vcVariant'] or 'KAKADU' in row['vcVariant']):
                    node_6.append('LUX')
                else:
                    node_6.append('UNKNOWN')
            elif (row['node_5'] == 'DIESEL'):
                if (('GX' in row['vcVariant'] and 'GXL' not in row['vcVariant'])):
                    node_6.append('GX')
                elif ('GXL' in row['vcVariant'] or 'ALTITUDE' in row['vcVariant'] or 'ANNIVERSARY' in row['vcVariant']):
                    node_6.append('GXL')
                elif ('VX' in row['vcVariant'] or 'KAKADU' in row['vcVariant']):
                    node_6.append('VX')
                elif ('SAHARA ' in row['vcVariant']):
                    node_6.append('SAHARA')
                else:
                    node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        else:
                    node_6.append('UNKNOWN')  

    elif (row['node_3'] == 'HILUX'):
        if (row['node_4'] == '4'):
            if (row['node_5'] == 'SR'):
                if (row['vcBodyStyle'] == 'C/CHAS'):
                    node_6.append('CC')
                elif ('C/CHAS' in row['vcBodyStyle'] and ('DUAL' in row['vcBodyStyle'] or 'DOUBLE' in row['vcBodyStyle'])):
                    node_6.append('DUAL CC')
                elif (('P/UP' in row['vcBodyStyle'] or 'UTILITY' in row['vcBodyStyle']) and ('DUAL' in row['vcBodyStyle'] or 'DOUBLE' in row['vcBodyStyle'])):
                    node_6.append('DUAL UTE')
                elif ('C/CHAS' in row['vcBodyStyle'] and 'X' in row['vcBodyStyle']):
                    node_6.append('X CAB CC')
                elif (('P/UP' in row['vcBodyStyle'] or 'UTILITY' in row['vcBodyStyle']) and 'X' in row['vcBodyStyle']):
                    node_6.append('X CAB UTE')
                elif ('P/UP' in row['vcBodyStyle'] or 'UTILITY' in row['vcBodyStyle']):
                    node_6.append('UTE')
                else:
                    node_6.append('UNKNOWN')
            
            elif (row['node_5'] == 'SR5'):
                node_6.append('UNKNOWN')
            elif (row['node_5'] == 'WORKMATE'):
                if (row['vcBodyStyle'] == 'C/CHAS'):
                    node_6.append('CC')
                elif (('P/UP' in row['vcBodyStyle'] or 'UTILITY' in row['vcBodyStyle']) and ('DUAL' in row['vcBodyStyle'] or 'DOUBLE' in row['vcBodyStyle'])):
                    node_6.append('DUAL UTE')
                else:
                    node_6.append('UNKNOWN')
            else:
                node_6.append('UNKNOWN')
        
        elif (row['node_4'] == '6'):
            if (row['node_5'] == 'SR'):
                node_6.append('UNKNOWN')
            elif (row['node_5'] == 'SR5'):
                node_6.append('UNKNOWN')
        else:
            node_6.append('UNKNOWN')
    
    elif (row['node_3'] == 'LC 70'):
        node_6.append ('UNKNOWN')
        
    elif (row['node_3'] == 'TARAGO'):
        node_6.append ('UNKNOWN')
    
    elif (row['node_3'] == 'HIACE'):
        node_6.append ('UNKNOWN')
        
    else:
        node_6.append ('UNKNOWN')
        
df_disposals['node_6'] = node_6



'Assign node 7 ##############################################################################################'

for index, row in df_disposals.iterrows():
    if (row['node_3'] == 'YARIS/PRIUS-C'):
        if (row['node_4'] == 'ASCENT'):
            if (row['node_5'] == 'ASCENT'):
                if (row['node_6'] == 'AUTO'):
                    if ('3D' in row['vcBodyStyle']):
                        node_7.append('3D HATCH')
                    elif ('5D' in row['vcBodyStyle']):
                        node_7.append('5D HATCH')
                    else:
                        node_7.append('UNKNOWN')
                elif (row['node_6'] == 'MANUAL'):
                    if ('3D' in row['vcBodyStyle']):
                        node_7.append('3D HATCH')
                    elif ('5D' in row['vcBodyStyle']):
                        node_7.append('5D HATCH')
                    else:
                        node_7.append('UNKNOWN')
                else:
                    node_7.append('UNKNOWN')
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'COROLLA/RUKUS'):
        if (row['node_4'] == 'ASCENT'):
            if (row['node_5'] == 'PETROL'):
                if (row['node_6'] == '4D SEDAN'):
                    node_7.append(row['clean_vcTransmissionType'])
                elif (row['node_6'] == '5D HATCH'):
                    node_7.append(row['clean_vcTransmissionType'])
                else:
                    node_7.append('UNKNOWN')
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'PRIUS'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'CAMRY/AURION'):
        if(row['node_4'] == '4'):
            if(row['node_5'] == 'HYBRID'):
                if(row['node_6'] == 'LUX'):
                    if ('HL' in row['vcVariant']):
                        node_7.append('HL')
                    elif ('SL' in row['vcVariant']):
                        node_7.append('SL')
                    elif ('LUX' in row['vcVariant']):
                        node_7.append('LUX')
                    else:
                        node_7.append('UNKNOWN')
                else:
                    if ('ATARA R' in row['vcVariant'] or 'ATARA S' in row['vcVariant'] or 'ATEVA' in row['vcVariant'] or 'CONQUEST' in row['vcVariant'] or 'ASCENT SPORT' in row['vcVariant']):
                        node_7.append('SPORT')
                    elif ( 'ALTISE' in row['vcVariant'] or 'RZ' in row['vcVariant'] or 'ASCENT' in row['vcVariant']):
                        node_7.append('ASCENT')
                    else:
                        node_7.append(row['vcVariant'])
            
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
                    
    elif (row['node_3'] == '86'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'RAV4/C-HR'):
        if(row['node_4'] == '4'):
            if(row['node_5'] == 'PETROL'):
                if(row['node_6'] == 'RAV4'):
                    node_7.append(row['clean_vcEngineSize'])
                else:
                    node_7.append('UNKNOWN')
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'KLUGER/FORTUNER/PRIUS V'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'PRADO'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'LC 200/ FJ CRUISER'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'HILUX'):
        if (row['node_4'] == '4'):
            if (row['node_5'] == 'SR'):
                if (row['node_6'] == 'DUAL CC'):
                    node_7.append(row['clean_vcEngineSize'])
                elif (row['node_6'] == 'DUAL UTE'):
                    node_7.append(row['clean_vcEngineSize'])
                else:
                    node_7.append('UNKNOWN')
            
            elif(row['node_5'] == 'WORKMATE'):
                if (row['node_6'] == 'CC'):
                    node_7.append(row['clean_vcFuelType'])
                elif (row['node_6'] == 'DUAL UTE'):
                    node_7.append(row['clean_vcFuelType'])
                else:
                    node_7.append('UNKNOWN')
            else:
                node_7.append('UNKNOWN')
        else:
            node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'LC 70'):
        node_7.append('UNKNOWN')
        
    elif (row['node_3'] == 'TARAGO'):
        node_7.append('UNKNOWN')
    
    elif (row['node_3'] == 'HIACE'):
        node_7.append('UNKNOWN')
    else:
        node_7.append('UNKNOWN')
df_disposals['node_7'] = node_7    



In [163]:
node_key = []

for index, row in df_disposals.iterrows():
    final_node = row['node_1'].strip() + ' ' + row['node_2'].strip() + ' ' + row['node_3'].strip() + ' ' + row['node_4'].strip() + ' ' + row['node_5'].strip() + ' ' + row['node_6'].strip() + ' ' + row['node_7'].strip() 
    temp = final_node.replace('UNKNOWN', '')
    node_key.append(temp.replace('  ', ' '))

df_disposals['node_key'] = node_key   

In [164]:
#df_disposals.head()
del df_vehicles["mRetailPrice"]
del df_disposals["vcSeries"]
del df_disposals["clean_vcVariant"]
del df_disposals["vcVariant_1"]
del df_disposals["vcVariant_2"]
del df_disposals["vcTransmissionType"]
del df_disposals["vcFuelType"]
del df_disposals["vcBodyStyle"]
del df_disposals["vcDriveType"]
del df_disposals["vcEngineSize"]
del df_disposals["node_1"]
del df_disposals["node_2"]
del df_disposals["node_3"]
del df_disposals["node_4"]
del df_disposals["node_5"]
del df_disposals["node_6"]
del df_disposals["node_7"]

In [165]:
#Reordering
df_disposals = df_disposals[['nvic','vcMake','vcModel', 'vcVariant','mRetailPrice','clean_vcBodyStyle','clean_vcTransmissionType','clean_vcDriveType','clean_vcFuelType','clean_vcEngineSize','cCylinders','dtReleaseDate','node_key' ]]
#Renaming
df_disposals.columns = ['cVehicleCode', 'vcMake','vcModel','vcVariant','mRetailPrice', 'vcBodyStyle', 'vcTransmissionType','vcDriveType','vcFuelType','vcEngineSize','cCylinders', 'dtReleaseDate','nodeKey']

In [167]:
cur = sql_conn2.cursor()
cur.execute("truncate table dbo.tblVehicleDataCleaned")
cur.commit()

In [168]:
quoted = urllib.parse.quote_plus('DRIVER={SQL Server};\
                           SERVER=aupanalytics02;\
                           DATABASE=RVMonthlyReporting;\
                           Trusted_Connection=yes')
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [169]:
df_disposals.to_sql('tblVehicleDataCleaned', engine, index=False, if_exists='append', chunksize=100, method='multi')

# Creating table for drop down lists

In [124]:
df_dropdown = df_vehicles[['vcMake','vcModel']].copy()
df_dropdown = df_dropdown.drop_duplicates()
df_dropdown = df_dropdown.pivot(columns='vcMake', values='vcModel').apply(lambda x: pd.Series(x.dropna().values)).fillna('')

vcMake,LEXUS,MAZDA,TOYOTA
0,LC500,CX-5,YARIS
1,LC500h (HYBRID),BT-50,LANDCRUISER PRADO
2,CT 200h. HYBRID,MAZDA2,HILUX
3,NX300,CX-3,LANDCRUISER 200
4,NX300h,CX-9,COASTER


## Writing out the Dataframes to excel 

In [127]:
wb = xw.Book('GFV management tool lite 14 xl wings.xlsm')
sheet = wb.sheets['Temp']
sheet.range('B:E').clear()
sheet["B1"].options(pd.DataFrame, header=1, index=False, expand='table').value = df_dropdown

sheet_2 = wb.sheets['Vehicle Data']
ranges = ''''2:''' + str(df_vehicles.shape[0]) + "'"
#sheet_2.range('2:4865').clear()
sheet_2["A2"].options(pd.DataFrame, header=0, index=False, expand='table').value = df_vehicles
#sheet.range('1:10').api.delete()
wb.save()

In [21]:
writer = pd.ExcelWriter('GFV management tool lite xl wings.xlsm', engine='openpyxl')
writer.book = openpyxl.load_workbook('GFV management tool lite.xlsm', keep_vba= True)
writer.sheets = {ws.title: ws for ws in writer.book.worksheets}

#worksheet_vehicle_data = writer.sheets['Vehicle Data']
#worksheet_temp = writer.sheets['Temp']
#worksheet_vehicle_data.delete_rows(1,5)

#df_vehicles.to_excel(writer, sheet_name= 'Vehicle Data', startrow=1, startcol=0, header=False, index=False)
df_dropdown.to_excel(writer, sheet_name = 'Temp', startrow = 0, startcol = 2, header=True, index=False)

workbook = writer.book
workbook.filename = 'GFV management tool lite xl wings.xlsm'

writer.save()
writer.close()
